## What is MLFlow and its Components

MLFLow is an open source platform to manage the ML lifecycle, including experimentation, reproducibility, deployment, and a central model registry. MLFLow currently offers four components:


<img src="mlflow.png">


https://www.mlflow.org/

_First thing first_
### Create Conda environment

##### run below commands in terminal but make sure conda is installed or use anaconda prompt which you will get as part of anaconda installation

1. `virtualenv --python=/usr/bin/python3 mlops-assignment` 
it will create a conda env named envname and install python version 3.9 and a ipykernel inside this environment

2. Activate the environment
`source mlops-assignment/bin/activate`



3. Now install all required dependencies to run this notebook

* `pip install pandas`
* `pip install numpy`
* `pip install scikit-learn`
* `pip install imblearn`
* `pip install matplotlib`
* `pip install mlflow`

Now open the notebook using below command: (from the anaconda prompt inside conda environment)

`jupyter notebook`


#### Make sure python is used from your newly created environment.

In [1]:
import sys
print(sys.executable)

/home/muzammil/anaconda3/bin/python


### Setting up MLFlow tracking server

We also specify artifact root and backend store URI. This makes it possible to store models.

After running this command tracking server will be accessible at `localhost:5010`

In [2]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5010 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

## Training

Now we can train models. See `train.py`.
It contains code from supervised machine learning tutorial; we added tracking metrics and model.

In [3]:
%%bash 
source mlflow_env_vars.sh
mlflow run . 

2023/05/08 09:38:54 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/08 09:38:54 INFO mlflow.projects.utils: === Created directory /tmp/tmpdh0rst0k for downloading remote URIs passed to arguments of type 'path' ===
2023/05/08 09:38:54 INFO mlflow.projects.backend.local: === Running command 'source /home/muzammil/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python train.py wine_dataset.csv' in run with ID 'a6a03171aecd460598bd06fd0c5638e5' === 
Successfully registered model 'sklearn_logistic_t'.
2023/05/08 09:38:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sklearn_logistic_t, version 1
Created version '1' of model 'sklearn_logistic_t'.


Run - Wine1 is logged to Experiment 


2023/05/08 09:38:56 INFO mlflow.projects: === Run (ID 'a6a03171aecd460598bd06fd0c5638e5') succeeded ===


## Serving model

Now that we trained our models we can go to *Models* page on MLFLow UI (http://localhost:5010/#/models).

Click *sklearn_logistics_T* on this page, choose a model and move it to *Production* stage.

The following cell will serve the model at localhost on port 5010.

In [8]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_logistic_t/Production -p 5011 --env-manager=conda 


In [12]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
echo $last_model_path



a6a03171aecd460598bd06fd0c5638e5


# Prediction

In [27]:
%%bash

data='[[1.69629634, -0.3559704, 1.204063, 0.12662813, 1.37800506, 0.81776615, 1.0941454, -0.28675063, 0.6769204, 0.54078152, 0.47263301, 0.04467224]]'


curl -v -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5011/invocations


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:5011...
* Connected to 127.0.0.1 (127.0.0.1) port 5011 (#0)
> POST /invocations HTTP/1.1
> Host: 127.0.0.1:5011
> User-Agent: curl/7.87.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 155
> 
} [155 bytes data]
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: gunicorn
< Date: Mon, 08 May 2023 04:49:51 GMT
< Connection: close
< Content-Type: application/json
< Content-Length: 20
< 
{ [20 bytes data]
100   175  100    20  100   155  13995   105k --:--:-- --:--:-- --:--:--  170k
* Closing connection 0


{"predictions": [0]}

# Thank You 